In [1]:
import numpy as np
import copy

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [3]:
def derivativeSigmoid(x):
    return x * (1 - x)

In [32]:
int2binary = {}
binary_dim = 8
# it will compute the largest number that is possible to represent with binary dim 8
largest_num = pow(2,binary_dim)

# it will create a table that will map an int to its binary form
binary = np.unpackbits(np.array([range(largest_num)], dtype=np.uint8).T, axis=1)

for i in range(largest_num):
    int2binary[i] = binary[i]

In [33]:
largest_num

256

In [34]:
binary

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [1, 1, 1, ..., 1, 0, 1],
       [1, 1, 1, ..., 1, 1, 0],
       [1, 1, 1, ..., 1, 1, 1]], dtype=uint8)

In [35]:
int2binary[11]

array([0, 0, 0, 0, 1, 0, 1, 1], dtype=uint8)

In [36]:
learning_rate = 0.1
input_neurons = 2
hidden_neurons = 16
output_neuron = 1

In [37]:
wh = 2 * np.random.random((input_neurons,hidden_neurons)) - 1
wout = 2 * np.random.random((hidden_neurons,output_neuron)) - 1
w_time = 2 * np.random.random((hidden_neurons,hidden_neurons)) - 1

wh_update = np.zeros_like(wh)
wout_update = np.zeros_like(wout)
wtime_update = np.zeros_like(w_time)

In [38]:
epochs = 100000
for i in range(epochs):
    a = np.random.randint(largest_num/2)
    b = np.random.randint(largest_num/2)
    
    a_binary = int2binary[a]
    b_binary = int2binary[b]
    
    c = a + b
    c_binary = int2binary[c]
    
    d = np.zeros_like(c_binary)
    
    overallError = 0
    
    layer_2_deltas = list()
    layer_1_values = list()
    layer_1_values.append(np.zeros(hidden_neurons))
    
    for position in range(binary_dim):
        x = np.array([[a_binary[binary_dim - position - 1], b_binary[binary_dim - position - 1]]])
        y = np.array([[c_binary[binary_dim - position - 1]]]).T
        
        # hidden layer
        layer_1 = sigmoid(np.dot(x,wh) + np.dot(layer_1_values[-1], w_time))
        
        # output layer
        layer_2 = sigmoid(np.dot(layer_1, wout))
        
        layer_2_error = y - layer_2
        slope_output = derivativeSigmoid(layer_2)
        layer_2_deltas.append((layer_2_error) * slope_output)
        
        overallError += np.abs(layer_2_error[0])
        
        d[binary_dim - position - 1] = np.round(layer_2[0][0])
        
        # store hidden layer to use it in next timestep
        layer_1_values.append(copy.deepcopy(layer_1))
        
    future_layer_1_delta = np.zeros(hidden_neurons)
    
    for position in range(binary_dim):
        X = np.array([[a_binary[position], b_binary[position]]])
        layer_1 = layer_1_values[-position - 1]
        prev_layer_1 = layer_1_values[-position - 2]
        
        # error at output layer
        layer_2_delta = layer_2_deltas[-position -  1]
        # error at hidden layer
        slope_hidden = derivativeSigmoid(layer_1)
        layer_1_delta = (future_layer_1_delta.dot(w_time.T)) + layer_2_delta.dot(wout.T) * slope_hidden
        
        # updating weights
        wout_update += np.atleast_2d(layer_1).T.dot(layer_2_delta)
        wtime_update += np.atleast_2d(prev_layer_1).T.dot(layer_1_delta)
        wh_update += X.T.dot(layer_1_delta)
        
        future_layer_1_delta = layer_1_delta
    
    wh += wh_update * learning_rate
    wout += wout_update * learning_rate
    w_time += wtime_update * learning_rate
    
    wout_update *= 0
    wh_update *= 0
    wtime_update *= 0
    
    if (i % 10000 == 0):
        print("Error",overallError)
        print("Pred",d)
        print("True",c_binary)
        out = 0
        for index, x in enumerate(reversed(d)):
            out += x*pow(2, index)
        print("True - {} + {} = {}".format(a,b,c))
        print("Pred - {} + {} = {}".format(a,b,out))
        print("#####################")
        
        

Error [3.88347099]
Pred [1 1 1 1 1 1 1 0]
True [0 1 1 1 0 0 1 1]
True - 91 + 24 = 115
Pred - 91 + 24 = 254
#####################
Error [4.]
Pred [0 0 0 0 0 0 0 0]
True [0 0 1 1 0 1 0 1]
True - 30 + 23 = 53
Pred - 30 + 23 = 0
#####################
Error [3.72349228]
Pred [1 1 1 1 1 1 1 1]
True [1 0 0 1 1 1 1 0]
True - 56 + 102 = 158
Pred - 56 + 102 = 255
#####################
Error [4.]
Pred [0 0 0 0 0 0 0 0]
True [1 0 0 0 1 0 1 1]
True - 26 + 113 = 139
Pred - 26 + 113 = 0
#####################
Error [4.]
Pred [1 1 1 1 1 1 1 1]
True [0 0 0 0 1 1 1 1]
True - 4 + 11 = 15
Pred - 4 + 11 = 255
#####################
Error [4.18943233]
Pred [0 0 0 0 0 0 0 0]
True [0 1 0 1 1 1 0 1]
True - 41 + 52 = 93
Pred - 41 + 52 = 0
#####################


KeyboardInterrupt: 